In [1]:
#My Dog Breed Classifier

In [10]:
!pip install -Uqq fastbook

In [3]:
from fastbook import *
from fastai.vision.widgets import *

/home/joseph/anaconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:81: UserWarning: 
    Found GPU0 GeForce GTX 650 Ti which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))
/home/joseph/anaconda3/lib/python3.8/site-packages/fastbook/__init__.py:18: UserWarning: Missing `graphviz` - please run `conda install fastbook`
  except ModuleNotFoundError: warn("Missing `graphviz` - please run `conda install fastbook`")


In [4]:
key = os.environ.get('AZURE_SEARCH_KEY' , '8300669115b74ee49c2fac22468582e2')

In [5]:
dogs= ['Labrador Retriever', 'German Shephard Dog','Poodle','Chihuahua', 'Golden Retriever','Yorkshire Terrier','Dachschund' , 'Beagle' , 'Boxer',
       'Minature Schnauzer', 'Shih Tzu', 'Bulldog', 'German Spitz', 'English Cocker Spaniel', 'Cavalier King Charles Spaniel','French Bulldog',
       'Pug', 'Rottweiler','English Setter','Maltese','English Springer Spaniel', 'German Shorthaired Pointer', 'Border Collie', 'Shetland Sheepdog',
       'Dobermann','West Highland White Terrier','Bernese Mountain Dog','Great Dane', 'Brittany Spaniel'] 
path = Path('dogs')
if not path.exists():
    path.mkdir()
    for o in dogs:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key , o ,max_images=50)
        download_images(dest, urls=results.attrgot('contentUrl'))

In [6]:
fns = get_image_files(path)
failed = verify_images(fns)


In [7]:
failed.map(Path.unlink)

(#14) [None,None,None,None,None,None,None,None,None,None...]

In [8]:
dogs_block = DataBlock(
    blocks = (ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms()
    )
dls=dogs_block.dataloaders(path)

Could not do one pass in your dataloader, there is something wrong in it


In [9]:
dls.train.show_batch(max_n=8, nrows=2, unique=True)

RuntimeError: CUDA error: no kernel image is available for execution on the device

In [ ]:
learn = cnn_learner(dls, resnet18, metrics = error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=3)

In [ ]:
help(plot_top_losses)

In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
btn_upload = widgets.FileUpload()
btn_upload

In [ ]:
img = PILImage.create(btn_upload.data[-1])

In [ ]:
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

In [ ]:
pred,pred_idx,probs = learn_inf.predict(img)
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

In [ ]:
btn_run = widgets.Button(description='Classify')
btn_run

In [ ]:
def on_click_classify(change):
  img= PILImage.create(btn_upload.data[-1])
  out_pl.clear_output()
  with out_pl: display(img.to_thumb(128,128))
  pred, pred_idx, probs = learn_inf.predict(img)
  lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [ ]:
btn_upload = widgets.FileUpload()
VBox([widgets.Label('Upload a picture of a dog!'), 
      btn_upload, btn_run, out_pl, lbl_pred])